In [1]:
from tqdm import tqdm
import sys
sys.path.append("../")
from threading import Thread
import numpy as np
import os
import time
import datetime
import pandas as pd
import pandas_market_calendars as mcal
from polygon import RESTClient
#from tqdm.notebook import tqdm
import time
import pytz
eastern = 'US/Eastern'

In [2]:
import threading
import time
import logging
import random
from multiprocessing import Queue
pd.options.mode.chained_assignment = None  # default='warn'


In [3]:
done = False
lck = threading.Lock()


In [4]:
trade_map={"I":"orig_id","x":"exchange","p":"price","i":"id","e":"correction","r":"trf_id","t":"sip_timestamp","y":"participant_timestamp","f":"trf_timestamp","q":"sequence_number","c":"conditions","s":"size","z":"tape"
}

trade_conditions = {'0': 'Regular', '1': 'Acquisition', '2': 'AveragePrice', '3': 'AutomaticExecution', '4': 'Bunched', '5': 'BunchSold', '6': 'CAPElection', '7': 'CashTrade', '8': 'Closing', '9': 'Cross', '10': 'DerivativelyPriced', '11': 'Distribution', '12': 'FormT(ExtendedHours)', '13': 'FormTOutOfSequence', '14': 'InterMarketSweep', '15': 'MarketCenterOfficialClose', '16': 'MarketCenterOfficialOpen', '17': 'MarketCenterOpening', '18': 'MarketCenterReOpenning', '19': 'MarketCenterClosing', '20': 'NextDay', '21': 'PriceVariation', '22': 'PriorReferencePrice', '23': 'Rule155Amex', '24': 'Rule127Nyse', '25': 'Opening', '26': 'Opened', '27': 'RegularStoppedStock', '28': 'ReOpening', '29': 'Seller', '30': 'SoldLast', '31': 'SoldLastStoppedStock', '32': 'SoldOutOfSequence', '33': 'SoldOutOfSequenceStoppedStock', '34': 'Split', '35': 'StockOption', '36': 'YellowFlag', '37': 'OddLot', '38': 'CorrectedConsolidatedClosePrice', '39': 'Unknown', '40': 'Held', '41': 'TradeThruExempt', '42': 'NonEligible', '43': 'NonEligible-extended', '44': 'Cancelled', '45': 'Recovery', '46': 'Correction', '47': 'AsOf', '48': 'AsOfCorrection', '49': 'AsOfCancel', '50': 'OOB', '51': 'Summary', '52': 'Contingent', '53': 'Contingent(Qualified)', '54': 'Errored'}


In [5]:
key = "qRFktbNL6A_WzaFhqR80IllApsmjPY6a"
def convert_conditions(obj):
    the_list = obj
    the_list = [str(intt) for intt in the_list]
    
    for l in the_list:
        if l.strip() == "":
            l = '-999'
    return ', '.join(the_list)
def ts_to_datetime(ts) -> str:
    return datetime.datetime.fromtimestamp(ts / 1000.0).strftime('%Y-%m-%d %H:%M')

def get_list_of_Exchanges():

    with RESTClient(key) as client:
        respex = client.stocks_equities_exchanges()

        stock_exchanges = respex.exchange
        exchange_list_dict = {}

        for i in stock_exchanges:
            the_dict = {}
            the_dict[i.i_d_of_the_exchange] = i.name
            #exchange_list_dict.append(the_dict)
            exchange_list_dict[i.i_d_of_the_exchange] = i.name
        return exchange_list_dict

def get_list_of_trade_conditions():

    with RESTClient(key) as client:
        respex = client.stocks_equities_condition_mappings()

        stock_exchanges = respex.exchange
        exchange_list_dict = {}

        for i in stock_exchanges:
            the_dict = {}
            the_dict[i.i_d_of_the_exchange] = i.name
            #exchange_list_dict.append(the_dict)
            exchange_list_dict[i.i_d_of_the_exchange] = i.name
        return exchange_list_dict
    
def update_condition_labels(keyFunction, values):
    k =  list(keyFunction[str(v)] for v in list(values))
    return ", ".join(k)

In [6]:
def etf_save_aggregate_data(symbol, df, date, url, group):
    #global lck
    
    file_path = os.path.join(url,'aggregated_1000_th'+str(group)+'.csv')
    
    if os.path.exists(file_path):
        #lck.acquire()
        
        
#         tmp_df = pd.read_csv(file_path)

#         tmpdf2 = tmp_df.append(df)
    
#         tmpdf2.to_csv(file_path, index=False)

        with open(file_path, mode = 'a') as f:
            df.to_csv(f, header=f.tell()==0)
        
        #lck.release()
    else:
        #df = df.iloc[: , 1:]
        df.to_csv(file_path)      


In [7]:
def calculate_group_vwap(df):
    
    df['pxs']= df['price'] * df['size']
    numerator=df['pxs'].sum()
    denom = df['size'].sum()
    vwap= numerator/denom
    return vwap

In [8]:
def data_preprocessing(trades, day):
    
    trades_data = trades
    
    for tr in trades_data:
        keys = tr.keys()
        if 'c' in keys:
            tr['c'] = convert_conditions(tr['c'])
    
    trades_df = pd.DataFrame(trades_data).rename(columns=trade_map)
    
    trades_df['date'] = day
    trades_df['raw_sip_timestamp'] = trades_df['sip_timestamp']
    trades_df['raw_participant_timestamp'] =trades_df['participant_timestamp']

    trades_df['sip_timestamp'] =pd.to_datetime(trades_df['sip_timestamp']).dt.tz_localize('utc').dt.tz_convert(eastern)
    trades_df['participant_timestamp'] =pd.to_datetime(trades_df['participant_timestamp']).dt.tz_localize('utc').dt.tz_convert(eastern)

    trades_df['hours'] = trades_df['sip_timestamp'].apply(lambda x: x.hour)
    trades_df['minutes'] = trades_df['sip_timestamp'].apply(lambda x: x.minute)
    trades_df['seconds'] = trades_df['sip_timestamp'].apply(lambda x: x.second)
    trades_df['time']=trades_df['sip_timestamp'].dt.time
    trades_df['time_string']=trades_df['sip_timestamp'].apply(lambda x: str(x.hour)+':'+str(x.minute)+':'+str(x.second))
    trades_df['conditions_string'] = trades_df['conditions']
    
    return trades_df

def get_trades_all(ticker, date, size):
    total_data = []
    lasttimestamp = 0
    while True:
        try:
            with RESTClient(key) as client:
                if lasttimestamp == 0:
                    resp = client.historic_trades_v2(ticker=str(ticker).upper(), date=date, limit=size, reverse=False)
                else:
                    resp = client.historic_trades_v2(ticker=str(ticker).upper(), date=date, timestamp = lasttimestamp, limit=size, reverse=False)
                result = resp.__dict__
                if "results" in result:
                    result = result['results']
                else:
                    break
                
                try:
                    lasttimestamp = result[-1]['t']
                    total_data.extend(result)
                except:
                    #print("error last timestamp - "+ ticker)
                    break
                    
                if len(result) == 50000:
                    continue
                else:
                    break
        except:
            #print("error get all - "+ ticker)
            break
                
    return total_data

In [9]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def get_valid_trading_days_for_range(start, end, n):
    nyse = mcal.get_calendar('NYSE')
    days = nyse.valid_days(start_date=start, end_date=end)
    days = [x.strftime('%Y-%m-%d') for x in days]
    
    if n > len(days):
        return list(days)
    nn = n-1
    return list(days)[-nn:]

In [10]:

def extract_qualified_symbols(df, plvl, advlvl):
    qualified = df[(df.p_lvl == plvl) & (df.adv_lvl == advlvl)]
    return qualified



def extract_opening_data(trades_df):
    trades_df['conditions_string'] = trades_df['conditions'].astype(str)
    open_= trades_df[trades_df.conditions_string.str.contains("16")]

    open_ = open_.sort_values('size',ascending=False)

    open_price= list(open_['price'])[0]
    open_size= list(open_['size'])[0]
    open_datetime = list(open_['sip_timestamp'])
    open_time = list(open_['sip_timestamp'].dt.time)[0]

    to_return = {
        'open_price':open_price,
        'open_time':open_time,
        'open_time_string':str(open_time.strftime("%H:%M:%S")),
        'open_size':open_size
    }
    
    return to_return

def extract_closing_data(trades_df):
    
    close= trades_df[trades_df.conditions_string.str.contains("15")]
    
    
    close = close.sort_values('size',ascending=False)
    close_price= list(close['price'])[0]
    close_size= list(close['size'])[0]
    close_datetime = list(close['sip_timestamp'])[0]
    close_time = list(close['sip_timestamp'].dt.time)[0]
    day = list(close['sip_timestamp'].dt.date)[0]

    to_return = {
        'close_price':close_price,
        'close_time': close_time,
        'close_time_string':str(close_time.strftime("%H:%M:%S")),
        'close_size':close_size,
        'day':day
    }
    return to_return

def aggregate_premarket_data(trades_df, open_time):
    premarket= trades_df[trades_df.time < open_time]
    pre_max_price = premarket.price.max()
    pre_min_price = premarket.price.min()
    pre_std_price = premarket.price.std()
    pre_median_price = premarket.price.quantile(0.5)
    pre_volume = premarket.size.sum()
    out = calculate_group_vwap(premarket)
    #df = df.assign(VWAP=((volume * price).cumsum() / vol.cumsum()).ffill())

    
    to_return = {
        'pre_max_price':pre_max_price,
        'pre_min_price': pre_min_price,
        'pre_std_price': pre_std_price,
        'pre_median_price': pre_median_price,
        'pre_volume':pre_volume,
        'pre_total_trades': len(premarket),
        'pre_vwap': out
    }
    
    return to_return

def aggregate_intraday_data(trades_df, open_time, close_time):
    regmarket= trades_df[(trades_df.time >= open_time) & (trades_df.time <= close_time)]
    reg_max_price = regmarket.price.max()
    reg_min_price = regmarket.price.min()
    reg_std_price = regmarket.price.std()
    reg_median_price = regmarket.price.quantile(0.5)
    reg_volume = regmarket.size.sum()
    out = calculate_group_vwap(regmarket)

    to_return = {
        'intraday_max_price':reg_max_price,
        'intraday_min_price':reg_min_price, 
        'intraday_std_price':reg_std_price,
        'intraday_median_price':reg_median_price, 
        'intraday_volume':reg_volume,
        'intraday_total_trades': len(regmarket),
        'intraday_vwap': out
    }
    return to_return    

def aggregate_postmarket_data(trades_df, close_time):
    postmarket= trades_df[trades_df.time > close_time]
    post_max_price = postmarket.price.max()
    post_min_price = postmarket.price.min()
    post_std_price = postmarket.price.std()
    post_median_price = postmarket.price.quantile(0.5)
    post_volume = postmarket.size.sum()
    out = calculate_group_vwap(postmarket)
    
    to_return = {
        'post_max_price':post_max_price,
        'post_min_price':post_min_price, 
        'post_std_price':post_std_price,
        'post_median_price':post_median_price, 
        'post_volume':post_volume,
        'post_total_trades': len(postmarket),
        'post_vwap': out
    }
    return to_return

In [11]:
def aggregate_data(resp_df, sym):
    final_result = []

    open_data_extract = extract_opening_data(resp_df)
    close_data_extract = extract_closing_data(resp_df)

    premarket_aggregate = aggregate_premarket_data(resp_df, open_data_extract['open_time'])
    postmarket_aggregate = aggregate_postmarket_data(resp_df, close_data_extract['close_time'])

    intraday_aggregate = aggregate_intraday_data(resp_df, open_data_extract['open_time'], close_data_extract['close_time'])

    merged_data = {**open_data_extract, **close_data_extract, **premarket_aggregate, **intraday_aggregate, **postmarket_aggregate, **{"symbol":sym}}

    final_result.append(merged_data)

    return pd.DataFrame(final_result)

In [12]:


valid_days = get_valid_trading_days_for_range('2020-06-01', '2021-05-20', 1000)

In [13]:
completed_symbols = []

In [14]:
def run_for_data(symbols, dates, url, tnum, thread_num):
    completed = []
    for symbol in tqdm(symbols, desc="Thread num: "+str(thread_num)):
        for day in dates:
            
            data = get_trades_all(symbol, day, 500000)
            try:
                df = data_preprocessing(data, day)
                aggregate = aggregate_data(df, symbol)
                etf_save_aggregate_data(symbol, aggregate, day, url, tnum)

            except:
                pass
                #print("Could not process symbol ", symbol,  " so skipping")
            time.sleep(0.08)
        completed.append(symbol)
        print("completed symbols", ": ".join(completed), '- thread num:', str(thread_num), ' - ', str(len(completed)), '/', str(len(symbols)))

##### URL PATH

In [15]:
url = os.path.join(os.getcwd(), 'output')


In [16]:
orders_df = pd.read_excel('orders_symbols.xlsx')
dollars_df = pd.read_excel('dollar_symbols.xlsx')
orders_df = orders_df.sort_values(by='share_filled', ascending=False)
dollars_df = dollars_df.sort_values(by='dollars_filled', ascending=False)
orders_sym_list = set(list(orders_df['symbol'].head(1000)))
dollars_sym_list = set(list(dollars_df['symbol'].head(1000)))
list_union = orders_sym_list.union(dollars_sym_list)


#### filter out 168

In [17]:
_orders_sym_list = set(list(orders_df['symbol'].head(2000)))
_dollars_sym_list = set(list(dollars_df['symbol'].head(200)))
_list_union = _orders_sym_list.union(_dollars_sym_list)

In [18]:
_list_union_list = list(_list_union)
list_union_list = list(list_union)

In [19]:
main_list = []
for item in _list_union_list:
    if item not in list_union_list:
        main_list.append(item)
    
        

In [20]:
print(len(list_union_list), len(_list_union_list), len(main_list))

1329 2001 753


In [21]:
total_threads = 7

In [22]:
chunk_per_thread = int(len(main_list) / total_threads)
chunk_per_thread

107

In [23]:
total_chunks = list(chunks(main_list, chunk_per_thread))
len(total_chunks)

8

In [24]:
file_group = 0

In [27]:
count = 0

In [28]:
allthreads = []
for l in range(len(total_chunks)):
    allthreads.append(Thread(target=run_for_data, args=(total_chunks[l], valid_days, url, file_group, l)))
        
    file_group = file_group + 1        



In [29]:
for allt in allthreads:
    allt.start()
    
for index, thread in enumerate(allthreads):
    logging.info("Main    : before joining thread %d.", index)
    thread.join()
    logging.info("Main    : thread %d done", index)

Thread num: 3:   0%|                                                                           | 0/107 [00:00<?, ?it/s]

Thread num: 0:   0%|                                                                           | 0/107 [00:00<?, ?it/s]


Thread num: 6:   0%|                                                                           | 0/107 [00:00<?, ?it/s]



Thread num: 7:   0%|                                                                             | 0/4 [00:00<?, ?it/s]




Thread num: 1:   0%|                                                                           | 0/107 [00:00<?, ?it/s]





Thread num: 5:   0%|                                                                           | 0/107 [00:00<?, ?it/s]






Thread num: 4:   0%|                                                                           | 0/107 [00:00<?, ?it/s]<ipython-input-7-f555145416aa>:6: RuntimeWarning: invalid value encountered in double_scalars
  vwap= numerator/denom
Thread num: 2:   1%|▌ 

completed symbols OMP - thread num: 2  -  1 / 107







Thread num: 1:   1%|▌                                                               | 1/107 [04:45<8:24:17, 285.45s/it]

completed symbols OSS - thread num: 1  -  1 / 107





Thread num: 6:   1%|▌                                                               | 1/107 [05:05<8:58:50, 305.00s/it]

...

completed symbols AACQ: CTK: SVC: EGAN: PSTI: SAEXQ: EB: ATEN: CDXC: PVL: SILV: SILJ: NRBO: CARA: RTLR: CXW: FRO: EAST: PING: JPST: EQT: IDN: KBSF: FCAU: CLDX: TRNE: NBL: MAIN: ELY: PRCP: BRN: HJLI: DENN: RSI: IHT: TRQ: KIN: UUP: AMS: PSXP: TGEN: PMM: OII: DXD: AEY: OMI: TMQ: ZUO: MMLP: EGLE: CPG: NEW: XLF: PCRFY: USAS: SREV: MCEP: IHRT: P055094: ONDSD: DTEA: PDAC: AGS: ICLK: USX: FIVG: WEN: FHN: WMC: CERC: SECO: ARR: BDSI: RRR: CX: SQNS: ECTM: VEON: SLCA: WTT: SLQT: AMCR: HKIB: DSSI: KBNT: REXN: CRK: RF: BFRA: JCS: PLYA: MTEM: XTLB: WPX: MRGE: COHN: OUST: GNPX: AFI: MOSY: SIX: DUO: LEAF: FAT: NCSM - thread num: 3  -  105 / 107



Thread num: 3:  99%|████████████████████████████████████████████████████████████▍| 106/107 [12:48:06<01:41, 101.35s/it]

completed symbols AACQ: CTK: SVC: EGAN: PSTI: SAEXQ: EB: ATEN: CDXC: PVL: SILV: SILJ: NRBO: CARA: RTLR: CXW: FRO: EAST: PING: JPST: EQT: IDN: KBSF: FCAU: CLDX: TRNE: NBL: MAIN: ELY: PRCP: BRN: HJLI: DENN: RSI: IHT: TRQ: KIN: UUP: AMS: PSXP: TGEN: PMM: OII: DXD: AEY: OMI: TMQ: ZUO: MMLP: EGLE: CPG: NEW: XLF: PCRFY: USAS: SREV: MCEP: IHRT: P055094: ONDSD: DTEA: PDAC: AGS: ICLK: USX: FIVG: WEN: FHN: WMC: CERC: SECO: ARR: BDSI: RRR: CX: SQNS: ECTM: VEON: SLCA: WTT: SLQT: AMCR: HKIB: DSSI: KBNT: REXN: CRK: RF: BFRA: JCS: PLYA: MTEM: XTLB: WPX: MRGE: COHN: OUST: GNPX: AFI: MOSY: SIX: DUO: LEAF: FAT: NCSM: TALO - thread num: 3  -  106 / 107



Thread num: 3: 100%|█████████████████████████████████████████████████████████████| 107/107 [12:49:02<00:00, 431.24s/it]

completed symbols AACQ: CTK: SVC: EGAN: PSTI: SAEXQ: EB: ATEN: CDXC: PVL: SILV: SILJ: NRBO: CARA: RTLR: CXW: FRO: EAST: PING: JPST: EQT: IDN: KBSF: FCAU: CLDX: TRNE: NBL: MAIN: ELY: PRCP: BRN: HJLI: DENN: RSI: IHT: TRQ: KIN: UUP: AMS: PSXP: TGEN: PMM: OII: DXD: AEY: OMI: TMQ: ZUO: MMLP: EGLE: CPG: NEW: XLF: PCRFY: USAS: SREV: MCEP: IHRT: P055094: ONDSD: DTEA: PDAC: AGS: ICLK: USX: FIVG: WEN: FHN: WMC: CERC: SECO: ARR: BDSI: RRR: CX: SQNS: ECTM: VEON: SLCA: WTT: SLQT: AMCR: HKIB: DSSI: KBNT: REXN: CRK: RF: BFRA: JCS: PLYA: MTEM: XTLB: WPX: MRGE: COHN: OUST: GNPX: AFI: MOSY: SIX: DUO: LEAF: FAT: NCSM: TALO: DRTT - thread num: 3  -  107 / 107


In [28]:
"j"

'j'